# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902005


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<00:59,  2.05s/it]

Rendering models:  10%|▉         | 3/31 [00:06<01:02,  2.23s/it]

Rendering models:  16%|█▌        | 5/31 [00:06<00:41,  1.59s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:29,  1.18s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  5.02it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  3.98it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  4.82it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  5.25it/s]

equidad1                              0.000276
not-logged-in-ca8f5ea40b91ce813161    0.000865
not-logged-in-20487e5808245554c355    0.000287
dayanakula                            0.001631
ElisabethB                            0.000349
not-logged-in-c0884513393674cd89f6    0.000562
Khaazel                               0.001030
kayleebug2017                         0.001804
MEMcKinney                            0.002498
r.garita.elizondo                     0.325579
clars915                              0.000304
not-logged-in-3a0f1883e4d5b2fcd92a    0.000324
not-logged-in-5f30b2bb9b7f9d49da41    0.005939
not-logged-in-73ac58943c8bd46f7314    0.002365
Planetme                              0.000731
not-logged-in-d9efd68e01383a25fcbf    0.001950
RaulE                                 0.000341
rhondacard                            0.002816
helios1235                            0.000396
Lavadude                              0.009542
not-logged-in-90294de164951e098d0b    0.002463
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())